In [22]:
import warnings
warnings.filterwarnings('ignore')

In [23]:
warnings
!pip install xgboost
!pip install catboost

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import warnings

In [6]:
df = pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [8]:
X = df.drop('math_score', axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [9]:
y = df['math_score']
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [10]:
num_feature = X.select_dtypes(exclude='object').columns
cat_feature = X.select_dtypes(include='object').columns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder',oh_transformer,cat_feature),
        ('StandardScaler',num_transformer,num_feature)
    ]
)

In [11]:
X = preprocessor.fit_transform(X)

In [12]:
X.shape

(1000, 19)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [17]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.mean(mse)
    r2_square = r2_score(true,predicted)
    return mae,rmse,r2_square

In [18]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "K nearest neighbor": KNeighborsRegressor(),
    "Support Vector Machine": SVR(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Ada Boost": AdaBoostRegressor(),
    "XG Boost": XGBRegressor(),
    "Cat Boost": CatBoostRegressor(verbose=False)
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    mae_train,rmse_train,r2_train = evaluate_model(y_train,y_train_pred)
    mae_test,rmse_test,r2_test = evaluate_model(y_test,y_test_pred)
    
    model_list.append(list(models.keys())[i])
    print(list(models.keys())[i])

    print("Model Training Set Evaluation:")
    print("-Mean Absolute Error : {:.4f}".format(mae_train))
    print("-Root Mean Square Error : {:.4f}".format(rmse_train))
    print("-r2 Score : {:.4f}".format(r2_train))

    print('-----------------------------------')
    
    print("Model Test Set Evaluation:")
    print("Mean Absolute Error : {:.4f}".format(mae_test))
    print("Root Mean Square Error : {:.4f}".format(rmse_test))
    print("r2 Score : {:.4f}".format(r2_test))

    r2_list.append(r2_test)

    print('='*35)
    print('\n')

Linear Regression
Model Training Set Evaluation:
-Mean Absolute Error : 4.2667
-Root Mean Square Error : 28.3349
-r2 Score : 0.8743
-----------------------------------
Model Test Set Evaluation:
Mean Absolute Error : 4.2148
Root Mean Square Error : 29.0952
r2 Score : 0.8804


Ridge Regression
Model Training Set Evaluation:
-Mean Absolute Error : 4.2650
-Root Mean Square Error : 28.3378
-r2 Score : 0.8743
-----------------------------------
Model Test Set Evaluation:
Mean Absolute Error : 4.2111
Root Mean Square Error : 29.0563
r2 Score : 0.8806


Lasso Regression
Model Training Set Evaluation:
-Mean Absolute Error : 5.2063
-Root Mean Square Error : 43.4784
-r2 Score : 0.8071
-----------------------------------
Model Test Set Evaluation:
Mean Absolute Error : 5.1579
Root Mean Square Error : 42.5064
r2 Score : 0.8253


K nearest neighbor
Model Training Set Evaluation:
-Mean Absolute Error : 4.5180
-Root Mean Square Error : 32.5953
-r2 Score : 0.8554
-----------------------------------
Mo

In [21]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Models','r2_Score']).sort_values(by=['r2_Score'],ascending=False)

,Models,r2_Score
1,Ridge Regression,0.880593
0,Linear Regression,0.880433
6,Random Forest,0.852142
9,Cat Boost,0.851632
7,Ada Boost,0.848854
8,XG Boost,0.827797
2,Lasso Regression,0.825320
3,K nearest neighbor,0.783898
5,Decision Tree,0.735964
4,Support Vector Machine,0.728600
